# Polymarket Basic Usage Example

This notebook demonstrates the basic functionality of the Polymarket exchange integration.

You'll learn how to:
- Initialize a public Polymarket client
- Fetch markets by event slug or URL
- Select a market by question text
- Resolve YES/NO token IDs by outcome name
- Fetch and normalize the order book
- Compute best bid/ask, mid price, top-N depth, and notional liquidity
- Print the order book in a classic trader-style vertical layout
- Fetch recent public trades
- Fetch and plot price history

## Setup

First, import the required modules and initialize the Polymarket exchange client.

In [ ]:
import dr_manhattan
from dr_manhattan.exchanges.polymarket import Polymarket

# Initialize public Polymarket client (no authentication required for public data)
exchange = Polymarket(config={})

## Fetching Markets by Event Slug or URL

You can fetch markets using either an event slug or a full Polymarket URL.

In [ ]:
# Example: Fetch markets by event slug
# You can use either:
# - Direct slug: "presidential-election-winner-2024"
# - Full URL: "https://polymarket.com/event/presidential-election-winner-2024"

event_slug = "presidential-election-winner-2024"

try:
    markets = exchange.fetch_markets_by_slug(event_slug)
    print(f"Found {len(markets)} markets for event '{event_slug}'")
    print()
    
    # Display market information
    for i, market in enumerate(markets):
        print(f"Market {i + 1}:")
        print(f"  Question: {market.question}")
        print(f"  Outcomes: {market.outcomes}")
        print(f"  Liquidity: ${market.liquidity:,.2f}")
        print(f"  Volume: ${market.volume:,.2f}")
        print()
except Exception as e:
    print(f"Error fetching markets: {e}")
    print("Try using a different event slug or URL from polymarket.com")

## Selecting a Market by Question Text

You can filter markets by searching for specific text in the question.

In [ ]:
# Let's fetch a simple binary market for demonstration
# We'll search for an active market using the search_markets function

markets = exchange.search_markets(
    limit=10,
    binary=True,
    closed=False,
    min_liquidity=1000,
)

if markets:
    # Select the first market for our example
    market = markets[0]
    print(f"Selected market: {market.question}")
    print(f"Outcomes: {market.outcomes}")
    print(f"Liquidity: ${market.liquidity:,.2f}")
    print(f"Current prices: {market.prices}")
else:
    print("No markets found. Trying to fetch any active binary market...")
    all_markets = exchange.fetch_markets({"active": True, "limit": 50})
    binary_markets = [m for m in all_markets if m.is_binary]
    if binary_markets:
        market = binary_markets[0]
        print(f"Selected market: {market.question}")
    else:
        print("No binary markets available for demonstration")

## Resolving Token IDs by Outcome Name

For binary markets, we can map outcome names (like 'Yes'/'No') to their token IDs.

In [ ]:
# Extract token IDs from market metadata
token_ids = exchange._extract_token_ids(market)

if token_ids and len(token_ids) >= 2:
    print("Token ID Mapping:")
    for i, outcome in enumerate(market.outcomes):
        if i < len(token_ids):
            print(f"  {outcome}: {token_ids[i]}")
    
    # Store for later use
    yes_token_id = token_ids[0]
    no_token_id = token_ids[1] if len(token_ids) > 1 else None
    
    print(f"\nWe'll use the '{market.outcomes[0]}' token (ID: {yes_token_id}) for orderbook analysis")
else:
    print("Token IDs not available in metadata. Fetching from CLOB API...")
    try:
        token_ids = exchange.fetch_token_ids(market.id)
        yes_token_id = token_ids[0] if token_ids else None
        no_token_id = token_ids[1] if len(token_ids) > 1 else None
        print(f"Token IDs: {token_ids}")
    except Exception as e:
        print(f"Error fetching token IDs: {e}")
        yes_token_id = None
        no_token_id = None

## Fetching and Normalizing the Order Book

Now let's fetch the order book and normalize the levels using our helper function.

In [ ]:
if yes_token_id:
    # Fetch raw orderbook
    orderbook = exchange.get_orderbook(yes_token_id)
    
    # Normalize orderbook levels using the helper function
    bids = Polymarket.normalize_orderbook_levels(orderbook.get('bids', []))
    asks = Polymarket.normalize_orderbook_levels(orderbook.get('asks', []))
    
    # Sort bids descending (highest first), asks ascending (lowest first)
    bids.sort(key=lambda x: x[0], reverse=True)
    asks.sort(key=lambda x: x[0])
    
    print(f"Orderbook for {market.outcomes[0]} token:")
    print(f"  Bids: {len(bids)} levels")
    print(f"  Asks: {len(asks)} levels")
else:
    print("Cannot fetch orderbook without token ID")
    bids = []
    asks = []

## Computing Order Book Metrics

Let's compute key orderbook metrics: best bid/ask, mid price, spread, and depth.

In [ ]:
if bids and asks:
    # Best bid and ask
    best_bid = bids[0][0]
    best_ask = asks[0][0]
    
    # Mid price
    mid_price = (best_bid + best_ask) / 2
    
    # Spread
    spread = best_ask - best_bid
    spread_bps = (spread / mid_price) * 10000 if mid_price > 0 else 0
    
    # Top-5 depth (cumulative size)
    def compute_depth(levels, n=5):
        return sum(size for price, size in levels[:n])
    
    bid_depth_5 = compute_depth(bids, 5)
    ask_depth_5 = compute_depth(asks, 5)
    
    # Notional liquidity (value in dollars for top-5)
    bid_liquidity_5 = sum(price * size for price, size in bids[:5])
    ask_liquidity_5 = sum(price * size for price, size in asks[:5])
    
    print("=" * 60)
    print("ORDER BOOK METRICS")
    print("=" * 60)
    print(f"Best Bid:        ${best_bid:.4f}")
    print(f"Best Ask:        ${best_ask:.4f}")
    print(f"Mid Price:       ${mid_price:.4f}")
    print(f"Spread:          ${spread:.4f} ({spread_bps:.2f} bps)")
    print()
    print(f"Top-5 Bid Depth:       {bid_depth_5:,.0f} shares")
    print(f"Top-5 Ask Depth:       {ask_depth_5:,.0f} shares")
    print()
    print(f"Top-5 Bid Liquidity:   ${bid_liquidity_5:,.2f}")
    print(f"Top-5 Ask Liquidity:   ${ask_liquidity_5:,.2f}")
    print("=" * 60)
else:
    print("Not enough orderbook data to compute metrics")

## Classic Trader-Style Order Book Display

Display the order book in a vertical layout like a trading terminal.

In [ ]:
def print_orderbook(bids, asks, depth=10):
    """Print orderbook in classic vertical trader layout."""
    print("\n" + "=" * 70)
    print(f"ORDER BOOK - {market.outcomes[0]} Token".center(70))
    print("=" * 70)
    print(f"{'BIDS':<25} {'PRICE':^15} {'ASKS':>25}")
    print(f"{'Size':>10} {'Total':>12} {' ':^15} {'Size':<10} {'Total':<12}")
    print("-" * 70)
    
    # Calculate cumulative sizes
    bid_cumulative = []
    total = 0
    for price, size in bids[:depth]:
        total += size
        bid_cumulative.append(total)
    
    ask_cumulative = []
    total = 0
    for price, size in asks[:depth]:
        total += size
        ask_cumulative.append(total)
    
    # Print asks (reversed, so highest ask at top)
    for i in range(min(depth, len(asks)) - 1, -1, -1):
        price, size = asks[i]
        cum = ask_cumulative[i]
        print(f"{' ':>23} {price:>7.4f} {size:>10.0f} {cum:>12.0f}")
    
    # Print spread line
    if bids and asks:
        spread = asks[0][0] - bids[0][0]
        print("-" * 70)
        print(f"Spread: ${spread:.4f}".center(70))
        print("-" * 70)
    
    # Print bids
    for i in range(min(depth, len(bids))):
        price, size = bids[i]
        cum = bid_cumulative[i]
        print(f"{size:>10.0f} {cum:>12.0f} {price:>7.4f}")
    
    print("=" * 70 + "\n")

if bids and asks:
    print_orderbook(bids, asks, depth=10)
else:
    print("No orderbook data available")

## Fetching Recent Public Trades

Let's fetch recent trades for this market to see trading activity.

In [ ]:
# Fetch recent public trades
try:
    trades = exchange.fetch_public_trades(
        market=market,
        limit=10,
        taker_only=True
    )
    
    if trades:
        print("=" * 80)
        print("RECENT TRADES")
        print("=" * 80)
        print(f"{'Time':<20} {'Side':<6} {'Price':>8} {'Size':>12} {'Outcome':<15}")
        print("-" * 80)
        
        for trade in trades[:10]:
            time_str = trade.timestamp.strftime('%Y-%m-%d %H:%M:%S')
            outcome_str = trade.outcome if trade.outcome else 'N/A'
            print(f"{time_str:<20} {trade.side:<6} {trade.price:>8.4f} {trade.size:>12.0f} {outcome_str:<15}")
        
        print("=" * 80)
    else:
        print("No recent trades found for this market")
        
except Exception as e:
    print(f"Error fetching trades: {e}")

## Fetching and Plotting Price History

Finally, let's fetch price history and create a simple visualization.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

if yes_token_id:
    try:
        # Fetch price history as a DataFrame
        price_history = exchange.fetch_price_history(
            market=market,
            outcome=0,  # First outcome (usually Yes)
            interval="1h",
            fidelity=100,
            as_dataframe=True
        )
        
        if not price_history.empty:
            # Create the plot
            fig, ax = plt.subplots(figsize=(12, 6))
            
            ax.plot(price_history['timestamp'], price_history['price'], 
                   linewidth=2, color='#2E86AB')
            
            # Format the plot
            ax.set_title(f"Price History: {market.question[:60]}...", 
                        fontsize=14, fontweight='bold', pad=20)
            ax.set_xlabel('Time', fontsize=12)
            ax.set_ylabel('Price ($)', fontsize=12)
            ax.grid(True, alpha=0.3, linestyle='--')
            
            # Format x-axis dates
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d %H:%M'))
            plt.xticks(rotation=45)
            
            # Add horizontal line at current price
            current_price = price_history['price'].iloc[-1]
            ax.axhline(y=current_price, color='red', linestyle='--', 
                      alpha=0.5, label=f'Current: ${current_price:.4f}')
            
            ax.legend(loc='best')
            
            plt.tight_layout()
            plt.show()
            
            # Print summary statistics
            print("\nPrice History Summary:")
            print(f"  Data points: {len(price_history)}")
            print(f"  Current price: ${current_price:.4f}")
            print(f"  High: ${price_history['price'].max():.4f}")
            print(f"  Low: ${price_history['price'].min():.4f}")
            print(f"  Average: ${price_history['price'].mean():.4f}")
        else:
            print("No price history data available")
            
    except Exception as e:
        print(f"Error fetching price history: {e}")
else:
    print("Cannot fetch price history without token ID")

## Summary

In this notebook, we've covered:

1. Initializing a public Polymarket client
2. Fetching markets by event slug or URL
3. Selecting markets and understanding their structure
4. Resolving token IDs for different outcomes
5. Fetching and normalizing order book data
6. Computing key order book metrics (best bid/ask, spread, depth, liquidity)
7. Displaying the order book in a trader-friendly format
8. Fetching recent public trades
9. Fetching and visualizing price history

This provides a solid foundation for building more advanced trading strategies and analytics on Polymarket.

### Next Steps

- Explore authenticated operations (placing orders, managing positions) by providing a private key
- Build real-time monitoring with WebSocket connections
- Implement custom trading strategies
- Analyze market inefficiencies and arbitrage opportunities